In [5]:
import math
from typing import List, Tuple, Union

class Point:
    def __init__(self, x: float, y: float, label: str = ""):
        self.x = x
        self.y = y
        self.label = label
    
    def to_svg(self) -> str:
        svg = f'<circle cx="{self.x}" cy="{self.y}" r="2" fill="black"/>'
        if self.label:
            svg += f'\n  <text x="{self.x + 5}" y="{self.y - 5}" font-family="Arial" font-size="12" fill="black">{self.label}</text>'
        return svg

class Line:
    def __init__(self, start: Point, end: Point):
        self.start = start
        self.end = end
    
    def to_svg(self) -> str:
        return f'<line x1="{self.start.x}" y1="{self.start.y}" x2="{self.end.x}" y2="{self.end.y}" stroke="black" stroke-width="1"/>'

class Circle:
    def __init__(self, center: Point, radius: float):
        self.center = center
        self.radius = radius
    
    def to_svg(self) -> str:
        return f'<circle cx="{self.center.x}" cy="{self.center.y}" r="{self.radius}" fill="none" stroke="black" stroke-width="1"/>'

class Triangle:
    def __init__(self, p1: Point, p2: Point, p3: Point):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
    
    def to_svg(self) -> str:
        points = f"{self.p1.x},{self.p1.y} {self.p2.x},{self.p2.y} {self.p3.x},{self.p3.y}"
        return f'<polygon points="{points}" fill="none" stroke="black" stroke-width="1"/>'

class Angle:
    def __init__(self, vertex: Point, point1: Point, point2: Point, radius: float = 20, color: str = "blue", label: str = ""):
        self.vertex = vertex
        self.point1 = point1
        self.point2 = point2
        self.radius = radius
        self.color = color
        self.label = label
    
    def _calculate_angle(self, p1: Point, vertex: Point, p2: Point) -> float:
        """Calcula o ângulo em radianos entre três pontos"""
        # Vetores do vértice para os outros pontos
        v1_x = p1.x - vertex.x
        v1_y = p1.y - vertex.y
        v2_x = p2.x - vertex.x
        v2_y = p2.y - vertex.y
        
        # Ângulos dos vetores
        angle1 = math.atan2(v1_y, v1_x)
        angle2 = math.atan2(v2_y, v2_x)
        
        return angle1, angle2
    
    def to_svg(self) -> str:
        angle1, angle2 = self._calculate_angle(self.point1, self.vertex, self.point2)
        
        # Garantir que o arco seja desenhado no sentido correto
        start_angle = angle1
        end_angle = angle2
        
        # Ajustar para o menor arco
        diff = end_angle - start_angle
        if diff > math.pi:
            end_angle -= 2 * math.pi
        elif diff < -math.pi:
            end_angle += 2 * math.pi
        
        # Converter para graus
        start_degrees = math.degrees(start_angle)
        end_degrees = math.degrees(end_angle)
        
        # Pontos do arco
        start_x = self.vertex.x + self.radius * math.cos(start_angle)
        start_y = self.vertex.y + self.radius * math.sin(start_angle)
        end_x = self.vertex.x + self.radius * math.cos(end_angle)
        end_y = self.vertex.y + self.radius * math.sin(end_angle)
        
        # Determinar se é um arco grande
        large_arc = 1 if abs(end_angle - start_angle) > math.pi else 0
        sweep = 1 if end_angle > start_angle else 0
        
        svg = f'<path d="M {start_x:.2f} {start_y:.2f} A {self.radius} {self.radius} 0 {large_arc} {sweep} {end_x:.2f} {end_y:.2f}" '
        svg += f'fill="none" stroke="{self.color}" stroke-width="1.5"/>'
        
        # Adicionar label se fornecido
        if self.label:
            # Posição do label no meio do arco
            mid_angle = (start_angle + end_angle) / 2
            label_radius = self.radius + 10
            label_x = self.vertex.x + label_radius * math.cos(mid_angle)
            label_y = self.vertex.y + label_radius * math.sin(mid_angle)
            svg += f'\n  <text x="{label_x:.2f}" y="{label_y:.2f}" font-family="Arial" font-size="10" fill="{self.color}" text-anchor="middle">{self.label}</text>'
        
        return svg

class Perpendicular:
    def __init__(self, triangle: Triangle, vertex: Point):
        self.triangle = triangle
        self.vertex = vertex
        self.perpendicular_line = self._calculate_perpendicular()
    
    def _calculate_perpendicular(self) -> Line:
        # Encontrar o lado oposto ao vértice
        if self.vertex == self.triangle.p1:
            opposite_start = self.triangle.p2
            opposite_end = self.triangle.p3
        elif self.vertex == self.triangle.p2:
            opposite_start = self.triangle.p1
            opposite_end = self.triangle.p3
        else:  # self.vertex == self.triangle.p3
            opposite_start = self.triangle.p1
            opposite_end = self.triangle.p2
        
        # Calcular o ponto médio do lado oposto
        mid_x = (opposite_start.x + opposite_end.x) / 2
        mid_y = (opposite_start.y + opposite_end.y) / 2
        mid_point = Point(mid_x, mid_y)
        
        return Line(self.vertex, mid_point)
    
    def to_svg(self) -> str:
        return f'<line x1="{self.perpendicular_line.start.x}" y1="{self.perpendicular_line.start.y}" x2="{self.perpendicular_line.end.x}" y2="{self.perpendicular_line.end.y}" stroke="red" stroke-width="1" stroke-dasharray="5,5"/>'

class Environment:
    def __init__(self, width: int = 800, height: int = 600):
        self.width = width
        self.height = height
        self.shapes: List[Union[Point, Line, Circle, Triangle, Perpendicular, Angle]] = []
    
    def add_point(self, x: float, y: float, label: str = "") -> Point:
        point = Point(x, y, label)
        self.shapes.append(point)
        return point
    
    def add_line(self, start: Point, end: Point) -> Line:
        line = Line(start, end)
        self.shapes.append(line)
        return line
    
    def add_circle(self, center: Point, radius: float) -> Circle:
        circle = Circle(center, radius)
        self.shapes.append(circle)
        return circle
    
    def add_triangle(self, p1: Point, p2: Point, p3: Point) -> Triangle:
        triangle = Triangle(p1, p2, p3)
        self.shapes.append(triangle)
        return triangle
    
    def add_angle(self, vertex: Point, point1: Point, point2: Point, radius: float = 20, color: str = "blue", label: str = "") -> Angle:
        angle = Angle(vertex, point1, point2, radius, color, label)
        self.shapes.append(angle)
        return angle
    
    def add_perpendicular(self, triangle: Triangle, vertex: Point) -> Perpendicular:
        perpendicular = Perpendicular(triangle, vertex)
        self.shapes.append(perpendicular)
        return perpendicular
    
    def generate_svg(self) -> str:
        svg_content = f'<svg width="{self.width}" height="{self.height}" xmlns="http://www.w3.org/2000/svg">\n'
        
        for shape in self.shapes:
            svg_content += f"  {shape.to_svg()}\n"
        
        svg_content += '</svg>'
        return svg_content
    
    def save_svg(self, filename: str):
        with open(filename, 'w') as f:
            f.write(self.generate_svg())

# Demonstração: Triângulo isósceles ABC com AB=AC
# Prova que ângulo ABC = ângulo BCA

print("PROVA: Em um triângulo isósceles ABC com AB=AC, os ângulos da base são iguais (∠ABC = ∠BCA)")
print("=" * 80)

# Configuração do ambiente
env = Environment(400, 300)

# Passo 1: Construir o triângulo isósceles ABC
print("\nPasso 1: Construindo o triângulo isósceles ABC onde AB = AC")

# Definindo os pontos para formar um triângulo isósceles
# A no topo, B e C na base, com AB = AC
A = env.add_point(200, 50, "A")   # Vértice superior
B = env.add_point(100, 200, "B")  # Vértice inferior esquerdo
C = env.add_point(300, 200, "C")  # Vértice inferior direito

# Verificando que AB = AC (distâncias iguais)
dist_AB = math.sqrt((A.x - B.x)**2 + (A.y - B.y)**2)
dist_AC = math.sqrt((A.x - C.x)**2 + (A.y - C.y)**2)
print(f"Distância AB = {dist_AB:.2f}")
print(f"Distância AC = {dist_AC:.2f}")
print(f"Confirmado: AB = AC (triângulo isósceles)")

# Adicionando o triângulo
triangle = env.add_triangle(A, B, C)

# Adicionando os ângulos da base para visualização
env.add_angle(B, A, C, radius=25, color="green", label="∠ABC")
env.add_angle(C, B, A, radius=25, color="green", label="∠BCA")
env.add_angle(A, B, C, radius=30, color="orange", label="∠BAC")

print("\nPasso 2: Adicionando a mediatriz do vértice A ao meio da base BC")

# Passo 2: Adicionar mediatriz de A ao ponto médio de BC
env.add_perpendicular(triangle, A)

# Calculando o ponto médio de BC para referência
mid_BC_x = (B.x + C.x) / 2
mid_BC_y = (B.y + C.y) / 2
mid_BC = env.add_point(mid_BC_x, mid_BC_y, "M")

print(f"Ponto médio M de BC: ({mid_BC_x}, {mid_BC_y})")
print("A mediatriz AM (linha vermelha tracejada) divide o triângulo em dois triângulos congruentes:")
print("- Triângulo ABM ≅ Triângulo ACM")
print("- Isso ocorre porque:")
print("  * AB = AC (dado - lados iguais)")
print("  * AM = AM (lado comum)")
print("  * BM = CM (M é ponto médio de BC)")
print("- Pela congruência LLL, os triângulos são congruentes")
print("- Portanto: ∠ABM = ∠ACM, ou seja, ∠ABC = ∠BCA")
print("- Os ângulos verdes mostram que ∠ABC = ∠BCA (ângulos da base iguais)")

# Gerando e exibindo o SVG
svg_output = env.generate_svg()
print(f"\nSVG gerado com {len(env.shapes)} elementos geométricos:")
print(svg_output)

print("\nCONCLUSÃO: A mediatriz do vértice A ao ponto médio da base BC demonstra")
print("que os ângulos da base de um triângulo isósceles são iguais. ∎")


PROVA: Em um triângulo isósceles ABC com AB=AC, os ângulos da base são iguais (∠ABC = ∠BCA)

Passo 1: Construindo o triângulo isósceles ABC onde AB = AC
Distância AB = 180.28
Distância AC = 180.28
Confirmado: AB = AC (triângulo isósceles)

Passo 2: Adicionando a mediatriz do vértice A ao meio da base BC
Ponto médio M de BC: (200.0, 200.0)
A mediatriz AM (linha vermelha tracejada) divide o triângulo em dois triângulos congruentes:
- Triângulo ABM ≅ Triângulo ACM
- Isso ocorre porque:
  * AB = AC (dado - lados iguais)
  * AM = AM (lado comum)
  * BM = CM (M é ponto médio de BC)
- Pela congruência LLL, os triângulos são congruentes
- Portanto: ∠ABM = ∠ACM, ou seja, ∠ABC = ∠BCA
- Os ângulos verdes mostram que ∠ABC = ∠BCA (ângulos da base iguais)

SVG gerado com 9 elementos geométricos:
<svg width="400" height="300" xmlns="http://www.w3.org/2000/svg">
  <circle cx="200" cy="50" r="2" fill="black"/>
  <text x="205" y="45" font-family="Arial" font-size="12" fill="black">A</text>
  <circle cx=

In [11]:
print("="*80)
print("PROBLEMA: Provar que a soma dos ângulos internos de um triângulo é 180°")
print("="*80)

# Criando um novo ambiente para esta demonstração
env2 = Environment()

print("\nPasso 1: Construindo um triângulo ABC qualquer")

# Definindo os pontos para formar um triângulo qualquer
A2 = env2.add_point(200, 80, "A")   # Vértice superior
B2 = env2.add_point(120, 220, "B")  # Vértice inferior esquerdo  
C2 = env2.add_point(320, 180, "C")  # Vértice inferior direito

# Adicionando o triângulo
triangle2 = env2.add_triangle(A2, B2, C2)

# Adicionando os ângulos internos do triângulo para visualização
# α está no vértice A (entre AB e AC)
env2.add_angle(A2, B2, C2, radius=25, color="red", label="α")
# β está no vértice B (entre BA e BC)
env2.add_angle(B2, A2, C2, radius=25, color="blue", label="β")
# γ está no vértice C (entre CA e CB)
env2.add_angle(C2, A2, B2, radius=25, color="green", label="γ")

print(f"Triângulo ABC com vértices:")
print(f"A = ({A2.x}, {A2.y})")
print(f"B = ({B2.x}, {B2.y})")
print(f"C = ({C2.x}, {C2.y})")

print("\nPasso 2: Traçando uma reta auxiliar paralela ao lado BC passando por A")

# Calculando a direção do lado BC
bc_dx = C2.x - B2.x
bc_dy = C2.y - B2.y

# Criando pontos para a reta paralela a BC passando por A
# Estendemos a reta para ambos os lados de A
D = env2.add_point(A2.x - bc_dx * 0.8, A2.y - bc_dy * 0.8, "D")
E = env2.add_point(A2.x + bc_dx * 0.8, A2.y + bc_dy * 0.8, "E")

# Adicionando a reta auxiliar DE (paralela a BC)
auxiliary_line = env2.add_line(D, E)

print(f"Reta auxiliar DE paralela a BC passando por A:")
print(f"D = ({D.x:.1f}, {D.y:.1f})")
print(f"E = ({E.x:.1f}, {E.y:.1f})")

print("\nPasso 3: Adicionando os ângulos formados pela reta auxiliar")

# Adicionando os ângulos alternos internos na reta auxiliar no ponto A
# β' está no ponto A, entre DA (reta auxiliar) e AB (alterno interno a β)
env2.add_angle(A2, D, B2, radius=20, color="blue", label="β'")
# γ' está no ponto A, entre AC e AE (reta auxiliar) (alterno interno a γ)
env2.add_angle(A2, C2, E, radius=20, color="green", label="γ'")

print("\nPasso 4: Análise dos ângulos formados")
print("\nDevido ao paralelismo entre DE e BC, temos:")
print("- ∠DAB = ∠ABC (ângulos alternos internos) → β' = β")
print("- ∠EAC = ∠ACB (ângulos alternos internos) → γ' = γ")
print("- ∠BAC é o ângulo interno do triângulo no vértice A → α")

print("\nPasso 5: Observando que os ângulos ∠DAB, ∠BAC e ∠EAC formam um ângulo raso")
print("Na reta DE, os três ângulos adjacentes somam 180°:")
print("β' + α + γ' = 180°")

print("\nPasso 6: Substituindo pelas equivalências dos ângulos alternos internos")
print("Como β' = β e γ' = γ, temos:")
print("β + α + γ = 180°")
print("Ou seja: ∠ABC + ∠BAC + ∠ACB = 180°")

print("\nPortanto, a soma dos ângulos internos do triângulo ABC é 180°!")

# Gerando e exibindo o SVG
svg_output2 = env2.generate_svg()
print(f"\nSVG gerado com {len(env2.shapes)} elementos geométricos:")
print(svg_output2)

print("\n" + "="*80)
print("CONCLUSÃO: A soma dos ângulos internos de qualquer triângulo é 180°")
print("Esta demonstração usa o fato de que ângulos alternos internos são iguais")
print("quando uma transversal corta duas retas paralelas. ∎")
print("="*80)


PROBLEMA: Provar que a soma dos ângulos internos de um triângulo é 180°

Passo 1: Construindo um triângulo ABC qualquer
Triângulo ABC com vértices:
A = (200, 80)
B = (120, 220)
C = (320, 180)

Passo 2: Traçando uma reta auxiliar paralela ao lado BC passando por A
Reta auxiliar DE paralela a BC passando por A:
D = (40.0, 112.0)
E = (360.0, 48.0)

Passo 3: Adicionando os ângulos formados pela reta auxiliar

Passo 4: Análise dos ângulos formados

Devido ao paralelismo entre DE e BC, temos:
- ∠DAB = ∠ABC (ângulos alternos internos) → β' = β
- ∠EAC = ∠ACB (ângulos alternos internos) → γ' = γ
- ∠BAC é o ângulo interno do triângulo no vértice A → α

Passo 5: Observando que os ângulos ∠DAB, ∠BAC e ∠EAC formam um ângulo raso
Na reta DE, os três ângulos adjacentes somam 180°:
β' + α + γ' = 180°

Passo 6: Substituindo pelas equivalências dos ângulos alternos internos
Como β' = β e γ' = γ, temos:
β + α + γ = 180°
Ou seja: ∠ABC + ∠BAC + ∠ACB = 180°

Portanto, a soma dos ângulos internos do triângul